In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import scdata as sc

from tools.backup import S3handler
from os.path import join
from scdata._config import config
config._out_level = 'DEBUG'
import pandas as pd
import re
import pprint
from IPython.display import display, HTML

pp = pprint.PrettyPrinter(indent=4)

s3handler = S3handler()
# s3handler.get_objects()
inventory_path = join(config.paths['inventory'], 'inventory', 'inventory.csv')
s3_inventory_path = 'inventory/deliveries/inventory.csv'
# Download it
s3handler.download(inventory_path, s3_inventory_path)
# Load files
with open(inventory_path, 'r') as inventory_csv:
    print (f'Loading inventory file from: {inventory_path}')
    inventory = pd.read_csv(inventory_csv).set_index('time')

In [ ]:
inventory.tail(6)

## Load

In [ ]:
destination = ''
batch = 1

In [ ]:
batch = str(batch)
test_names = list(set(list(inventory.loc[(inventory['destination'] == destination), 'test'])))

data_st = dict()
print ('--------')

for ntest in test_names:
    data_st[ntest] = dict()
    data_st[ntest]['kits'] = [re.sub('https://smartcitizen.me/kits/','',x) 
                              for x in list(inventory.loc[(inventory['test'] == ntest), 'platform_url'].values)]

    data_st[ntest]['min_date'] = list(set(list(inventory.loc[(inventory['destination'] == destination) 
                                                             & (inventory['test'] == ntest)
                                                             & (inventory['validated'] == 'y'), 
                                                             'min_validation_date'].dropna().values)))

    data_st[ntest]['max_date'] = list(set(list(inventory.loc[(inventory['destination'] == destination)
                                                             & (inventory['test'] == ntest)
                                                             & (inventory['validated'] == 'y'), 
                                                             'max_validation_date'].dropna().values)))

    print (f'TEST: {ntest}')
    print (data_st[ntest])    
    data_st[ntest]['test'] = sc.Test(ntest)
    data_st[ntest]['test'].purge()    
    data_st[ntest]['test'].load()
    print ('--------')

## Dispersion Analysis

In [ ]:
for ntest in test_names:
    data_st[ntest]['test'].get_common_channels()
    print('\nDispersion Analysis Summary:\n')
    pp.pprint(data_st[ntest]['test'].dispersion_analysis())

#### Static

In [ ]:
for ntest in test_names:
    [data_st[ntest]['test'].ts_dispersion_plot(channel = ch) for ch in data_st[ntest]['test'].common_channels];

#### Interactive

In [ ]:
for ch in data_st[ntest]['test'].common_channels:
    display(data_st[ntest]['test'].ts_dispersion_uplot(channel = ch, formatting = {'width': 1000, 'height': 300, 'padding-bottom': 600}))

### Fill details

In [ ]:
ntest = ''
data_st[ntest]['test'].descriptor['notes'] = '* 2021-04-26 17:44 replaced urban board of 2C47 (13997)'
data_st[ntest]['test'].__update_descriptor__()

In [ ]:
min_date_val = '2020-07-28'
max_date_val = '2020-08-03'

In [ ]:
failing_kits   = {'13084': ['PM_SENSOR'],
                  '13087': ['PM_SENSOR']
                 }

In [ ]:
delivered_kits = {'13077': {'replacement': 'none', 'min_date': min_date_val, 'max_date': max_date_val}, 
                  '13079': {'replacement': 'none', 'min_date': min_date_val, 'max_date': max_date_val},
                  '13081': {'replacement': 'none', 'min_date': min_date_val, 'max_date': max_date_val}, 
                  '13082': {'replacement': 'none', 'min_date': min_date_val, 'max_date': max_date_val},
                  '13086': {'replacement': 'none', 'min_date': min_date_val, 'max_date': max_date_val},
                  '13089': {'replacement': 'none', 'min_date': min_date_val, 'max_date': max_date_val}
                 }

In [ ]:
for kit in failing_kits.keys():
    inventory.loc[inventory.platform_url == f'https://smartcitizen.me/kits/{kit}', 'validated'] = 'n'
    inventory.loc[inventory.platform_url == f'https://smartcitizen.me/kits/{kit}', 'replacement'] = failing_kits[kit]

for kit in delivered_kits.keys():
    inventory.loc[inventory.platform_url == f'https://smartcitizen.me/kits/{kit}', 'validated'] = 'y'
    inventory.loc[inventory.platform_url == f'https://smartcitizen.me/kits/{kit}', 'replacement'] = delivered_kits[kit]['replacement']
    inventory.loc[inventory.platform_url == f'https://smartcitizen.me/kits/{kit}', 'min_validation_date'] = delivered_kits[kit]['min_date']    
    inventory.loc[inventory.platform_url == f'https://smartcitizen.me/kits/{kit}', 'max_validation_date'] = delivered_kits[kit]['max_date']

#### Upload inventory

In [ ]:
inventory.to_csv(inventory_path)

In [ ]:
s3handler.upload(inventory_path, s3_inventory_path) #expiration in seconds

## Report

### Description

In [ ]:
nok_devices = inventory.loc[(inventory['destination'] == destination)\
                                         & (inventory['batch'] == batch)\
                                         & (inventory['validated'] == 'n'),
                                         'platform_url']
print (f'NOK Devices ({len(nok_devices)})')
display (nok_devices)

ok_devices = inventory.loc[(inventory['destination'] == destination)\
                                  & (inventory['batch'] == str(batch))\
                                  & (inventory['validated'] == 'y'),
                                  ['platform_url', 'tested','validated', 
                                   'min_validation_date', 'max_validation_date']]
print (f'OK Devices ({len(ok_devices)})')
display(ok_devices)

In [ ]:
t = sc.Test(f'{destination}_JOIN_DELIVERY')

t.descriptor['author']= config.author
t.descriptor['type_test']= 'indoor'
t.descriptor['comment']= f'Join test for delivery of batch #{batch} to {destination}'

t.descriptor['delivery'] = {
    'destination': destination,
    'batch': str(batch),
    'devices': HTML(ok_devices.to_html())
}

t.create(force=True)

### Timeseries Grid

In [ ]:
for ntest in test_names:
    fig = data_st[ntest]['test'].ts_dispersion_grid()
    t.add_content(title=f'Timeseries test {ntest}', figure = fig)

### Scatter grid

In [ ]:
for ntest in test_names:
    fig = data_st[ntest]['test'].scatter_dispersion_grid()
    t.add_content(title=f'Scatter test {ntest}', figure = fig)

## Upload

In [ ]:
report_path = join(t.path, 'export', f'{t.full_name}.html')
t.to_html('Validation report', template = 'delivery_template.html', devices_summary = False);
t.descriptor['report'] = report_path
t.__update_descriptor__()
!open $report_path

In [ ]:
s3_report_path = f'reports/{t.full_name}.html'
url = s3handler.upload(report_path, s3_report_path, expiration=14*24*3600) #expiration in seconds